# Solución del desafío LATAM - Felipe Moya González

<br>

---

<br>

## Librerías necesarias

In [135]:
# Importamos las librerías a utilizar
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
import seaborn as sns
from janitor import clean_names
pd.set_option("display.max_columns", None)

Dejaremos una semilla para que los resultados obtenidos sean reproducibles.

In [136]:
np.random.seed(201212189)

<br>

---

<br>

## Carga de datos

Lo primero que debemos hacer es cargar los datos. Cargamos `Vlo-I` y `Vlo-O` como texto, pues hay algunos valores que contienen la letra P.

In [137]:
# Definimos variables para las rutas de los archivo
input_path = "data/input"
output_path = "data/output" 

# Cargamos los datos entregados
vuelos = pd.read_csv(os.path.join(input_path, "dataset_SCL.csv"), dtype={1: object, 6: object}).clean_names()

# Definimos la cantidad de filas y de columnas de la base
filas, columnas = vuelos.shape

<br>

---

<br>

## Limpieza de datos y análisis exploratorio

Veamos alguna información de los datos:

In [138]:
# Veamos algunas observaciones de los datos
vuelos.head()

,fecha_i,vlo_i,ori_i,des_i,emp_i,fecha_o,vlo_o,ori_o,des_o,emp_o,dia,mes,ano,dianom,tipovuelo,opera,siglaori,siglades
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,1,1,2017,Domingo,I,American Airlines,Santiago,Miami
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,2,1,2017,Lunes,I,American Airlines,Santiago,Miami
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,3,1,2017,Martes,I,American Airlines,Santiago,Miami
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,4,1,2017,Miercoles,I,American Airlines,Santiago,Miami
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,5,1,2017,Jueves,I,American Airlines,Santiago,Miami


In [139]:
# Mostramos las dimensiones de los datos.
print(f"\n La base contiene {filas} observaciones y {columnas} columnas.")


 La base contiene 68206 observaciones y 18 columnas.


In [140]:
# Veamos si todas las variables tienen el tipo adecuado
vuelos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68206 entries, 0 to 68205
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   fecha_i    68206 non-null  object
 1   vlo_i      68206 non-null  object
 2   ori_i      68206 non-null  object
 3   des_i      68206 non-null  object
 4   emp_i      68206 non-null  object
 5   fecha_o    68206 non-null  object
 6   vlo_o      68205 non-null  object
 7   ori_o      68206 non-null  object
 8   des_o      68206 non-null  object
 9   emp_o      68206 non-null  object
 10  dia        68206 non-null  int64 
 11  mes        68206 non-null  int64 
 12  ano        68206 non-null  int64 
 13  dianom     68206 non-null  object
 14  tipovuelo  68206 non-null  object
 15  opera      68206 non-null  object
 16  siglaori   68206 non-null  object
 17  siglades   68206 non-null  object
dtypes: int64(3), object(15)
memory usage: 9.4+ MB


Notamos que no todas las columnas tienen el tipo adecuado, así que ahora las pasamos al tipo adecuado: 

In [141]:
# Convertimos las variables en su tipo
vuelos["fecha_i"] = pd.to_datetime(vuelos["fecha_i"], format="%Y-%m-%d")
vuelos["fecha_o"] = pd.to_datetime(vuelos["fecha_o"], format="%Y-%m-%d")
 
# Pasamos la variable "tipovuelo" a dummie por su naturaleza categórica
vuelos = pd.get_dummies(vuelos, prefix="tipovuelo", columns=["tipovuelo"])

#### Valores faltantes:

Ahora revisaremos si es que hay valores faltantes en alguna de las columnas, para luego ver como encargarse de ellos.

In [142]:
# Valores faltantes por columna
vuelos.isna().sum()

fecha_i        0
vlo_i          0
ori_i          0
des_i          0
emp_i          0
fecha_o        0
vlo_o          1
ori_o          0
des_o          0
emp_o          0
dia            0
mes            0
ano            0
dianom         0
opera          0
siglaori       0
siglades       0
tipovuelo_I    0
tipovuelo_N    0
dtype: int64

Notamos que la única variable con valores faltantes es `vlo-o`. Veamos esa fila para tratar de entender porqué no tiene valor.

In [143]:
# Revisión fila con valor faltante
vuelos[vuelos["vlo_o"].isna()]

,fecha_i,vlo_i,ori_i,des_i,emp_i,fecha_o,vlo_o,ori_o,des_o,emp_o,dia,mes,ano,dianom,opera,siglaori,siglades,tipovuelo_I,tipovuelo_N
6068,2017-01-19 11:00:00,200,SCEL,SPJC,LAW,2017-01-19 11:03:00,NaN,SCEL,SPJC,56R,19,1,2017,Jueves,Latin American Wings,Santiago,Lima,1,0


Es un vuelo de Santiago a Lima, que tiene un número de vuelo programado, pero no un número de vuelo real. Por lo tanto, podemos reemplazar el valor faltante por el valor de la columna `vlo-i`, es decir con el número de vuelo programado.

In [144]:
# Reemplazamos por el número de vuelo programado
vuelos.loc[vuelos["vlo_o"].isna(), "vlo_o"] = vuelos.loc[vuelos["vlo_o"].isna(), "vlo_i"]

<br>

---

<br>

# 1.) 

## Distribución de los datos:

Observando cada columna, podemos ver que la mayorías de las variables son de texto, por lo que no podemos asociarle alguna distribución estadística. Además las que son numéricas son códigos de vuelo o fechas, por lo que tampoco podemos asociarle alguna distribución estadística. Lo que sí podemos hacer es ver la distribución de los datos en términos de frecuencia, para ver si hay valores que se repitan más que otros.

In [122]:
vuelos

,fecha_i,vlo_i,ori_i,des_i,emp_i,fecha_o,vlo_o,ori_o,des_o,emp_o,dia,mes,ano,dianom,opera,siglaori,siglades,tipovuelo_I,tipovuelo_N
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,1,1,2017,Domingo,American Airlines,Santiago,Miami,1,0
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,2,1,2017,Lunes,American Airlines,Santiago,Miami,1,0
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,3,1,2017,Martes,American Airlines,Santiago,Miami,1,0
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,4,1,2017,Miercoles,American Airlines,Santiago,Miami,1,0
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,5,1,2017,Jueves,American Airlines,Santiago,Miami,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68201,2017-12-22 14:55:00,400,SCEL,SPJC,JAT,2017-12-22 15:41:00,400.0,SCEL,SPJC,JAT,22,12,2017,Viernes,JetSmart SPA,Santiago,Lima,1,0
68202,2017-12-25 14:55:00,400,SCEL,SPJC,JAT,2017-12-25 15:11:00,400.0,SCEL,SPJC,JAT,25,12,2017,Lunes,JetSmart SPA,Santiago,Lima,1,0
68203,2017-12-27 14:55:00,400,SCEL,SPJC,JAT,2017-12-27 15:35:00,400.0,SCEL,SPJC,JAT,27,12,2017,Miercoles,JetSmart SPA,Santiago,Lima,1,0
68204,2017-12-29 14:55:00,400,SCEL,SPJC,JAT,2017-12-29 15:08:00,400.0,SCEL,SPJC,JAT,29,12,2017,Viernes,JetSmart SPA,Santiago,Lima,1,0


## Ideas importantes

## Conclusiones

<br>

---

<br>

# 2.) 

## Creación de columnas adicionales

Ahora crearemos las columnas adicionales que se piden, para luego exportarlas a un archivo csv.

In [182]:
# high_season
vuelos["high_season"] = vuelos["fecha_i"].apply(lambda x: 
1 if (x.month == 12 and x.day >= 15) or \
    x.month == 1 or x.month == 2 or \
        (x.month == 3 and x.day <= 3) or \
            (x.month == 7 and x.day >= 15) or \
                (x.month == 9 and x.day >= 11) else 0)

# min_diff
vuelos["min_diff"] = ((vuelos["fecha_o"] - vuelos["fecha_i"]).dt.seconds / 60).astype(int)

# delay_15
vuelos["delay_15"] = vuelos["min_diff"].apply(lambda x: 1 if x >= 15 else 0)

# period_day
vuelos["period_day"] = vuelos["fecha_i"].apply(lambda x: "Morning" if (x.hour < 12 and x.hour >= 5) else "Afternoon" if (x.hour < 19 and x.hour >= 12) else "Night")

<br>

---

<br>

# 3.)

## Relación entre algunas variables y el retraso del vuelo

<br>

---

<br>

# 4.)

## Entrenamiento de modelos

<br>

---

<br>

# 5.)

## Desempeño de los modelos

<br>

---

<br>